In [1]:
import polars as pl
import altair as alt

In [2]:
import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [3]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1993
).filter(
    pl.col("komora_komplet").str.contains('Poslanecká') | pl.col('komora_komplet').str.contains('PČR, PS')
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).with_columns(
    pl.when(pl.col("soubor") == "1996ps_stenprot_006schuz_s006024.htm").then(pl.lit(2006)).otherwise(pl.col('obdobi')).alias('obdobi')
).with_columns(
    pl.col("text").str.split("(").list.slice(1).list.eval(
        pl.element().str.split(')').list.first()
    ).list.eval(
        pl.element().str.to_lowercase().str.strip_chars(), parallel=True
    ).alias("poznamky_text")
).with_columns(
    pl.col("mluvci").str.split("(").list.slice(1).list.eval(
        pl.element().str.split(')').list.first()
    ).list.eval(
        pl.element().str.to_lowercase().str.strip_chars(), parallel=True
    ).alias("poznamky_mluvci")
).drop_nulls(
    subset=['datum']
).collect()

In [4]:
df.sample(10)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Ministr kultury ČR Jiří Besser""","""2010ps_stenprot_007schuz_s007153.htm""","""/sqw/detail.sqw?id=5896""","""Poté se bude pracovat s tou valtickou částí a samozřejmě je tam možnost zařazení v IOP 5.1.c) tak, jak skutečně tato aktivita je Národním památkovým ú…",1,"""ps""",2010,7,153,2010-11-04,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""7. schůze""",true,true,false,"""5896""",143,[],[]
"""Poslanec Karel Sehoř""","""1996ps_stenprot_015schuz_s015061.htm""","""/sqw/detail.sqw?id=48&o=2""","""Konečně bod G4. Předkladatel nesouhlasí, zpravodaj nesouhlasí.""",87,"""ps""",1996,15,61,1997-10-23,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""15. schůze""",true,true,true,"""48&o=2""",6,[],[]
"""Předseda PSP Jan Hamáček""","""2013ps_stenprot_047schuz_s047144.htm""","""/sqw/detail.sqw?id=5462""","""Dále do podrobné rozpravy? Pokud se nikdo nehlásí, tak i podrobnou rozpravu končím. Závěrečné slovo, pane ministře. Není třeba. Pan zpravodaj? Také ne…",6,"""ps""",2013,47,144,2016-05-31,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""47. schůze""",true,true,true,"""5462""",139,[],[]
"""Místopředseda PSP Petr Gazdík""","""2013ps_stenprot_008schuz_s008007.htm""","""/sqw/detail.sqw?id=5914""","""Děkuji panu předsedovi. Ptám se, kdo další se hlásí do obecné rozpravy. (Nikdo se nehlásí.) Jestliže nikdo, končím obecnou rozpravu. Protože byla podá…",13,"""ps""",2013,8,7,2014-04-29,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""8. schůze""",true,true,true,"""5914""",52,"[""nikdo se nehlásí.""]",[]
"""Poslanec Marek Ženíšek""","""2021ps_stenprot_035schuz_s035185.htm""","""/sqw/detail.sqw?id=6154""","""Děkuji za slovo, pane místopředsedo. Zahraniční výbor projednal daný tisk na své schůzi 1. září tohoto roku a nutno říct, že v diskusi většina členů v…",2,"""ps""",2021,35,185,2022-09-27,"""PČR, PS 2021-...""","""35. schůze""",true,true,true,"""6154""",146,[],[]
"""Předsedkyně PSP Miroslava Němcová""","""2010ps_stenprot_014schuz_s014038.htm""","""/sqw/detail.sqw?id=401""","""Děkuji. Ptám se na další přihlášku. Prosím pan kolega Josef Šenfeld má slovo, prosím.""",6,"""ps""",2010,14,38,2011-03-16,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""14. schůze""",true,true,true,"""401""",13,[],[]
"""Poslanec Petr Bendl""","""2017ps_stenprot_034schuz_s034138.htm""","""/sqw/detail.sqw?id=346""","""Děkuji za slovo. Já jsem chtěl podotknout jako člověk, který se dlouhodobě zabývá územními plány a debatami nad územními plány, kde téměř všude, kde b…",11,"""ps""",2017,34,138,2019-09-13,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""34. schůze""",true,true,true,"""346""",72,[],[]
"""Místopředseda PSP Tomio Okamura""","""2017ps_stenprot_104schuz_s104271.htm""","""/sqw/detail.sqw?id=6105""","""Já vás požádám o klid, aby se paní poslankyně mohla v klidu vyjádřit, protože se nemůže soustředit. No, paní poslankyně chce asi naznačit tu diskusi p…",4,"""ps""",2017,104,271,2021-06-16,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""104. schůze""",true,true,true,"""6105""",27,[],[]
"""Místopředsedkyně PSP Miroslava Němcová""","""2002ps_stenprot_008schuz_s008313.htm""","""/sqw/detail.sqw?id=401""","""To bylo vystoupení pana poslance Martina Římana, za které mu děkuji. S faktickou poznámkou se hlásí pan poslanec Laštůvka. Prosím.""",2,"""ps""",2002,8,313,2002-12-18,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""8. schůze""",true,true,true,"""401""",19,[],[]


In [5]:
df.group_by("pocet_slov").len().sort(by="pocet_slov")

pocet_slov,len
u32,u32
0,1353
1,7804
2,3233
3,4402
4,9248
5,7185
6,7198
7,7759
8,8472


In [6]:
df.select(pl.col("pocet_slov")).median()

pocet_slov
f64
45.0


In [7]:
df.select(pl.col("pocet_slov")).quantile(0.25)

pocet_slov
f64
18.0


In [8]:
slov_po_roce = df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('pocet_slov').sum())

In [9]:
len(df)

585161

In [10]:
df.columns

['mluvci',
 'soubor',
 'mluvci_id',
 'text',
 'poradi',
 'komora',
 'obdobi',
 'schuze',
 'cast',
 'datum',
 'komora_komplet',
 'schuze_komplet',
 'prepsano',
 'autorizovano',
 'mluvci_jisty',
 'id',
 'pocet_slov',
 'poznamky_text',
 'poznamky_mluvci']

In [11]:
def zavorka(retezec, doslova=True, frejm=df, epocha="datum"):

    frejm = frejm.with_columns(
        (pl.col('komora').cast(str) + pl.lit('-') + pl.col('obdobi').cast(str) + pl.lit('-') + pl.col('schuze').cast(str) + pl.lit('-') + pl.col('cast').cast(str) + pl.lit('-') + pl.col('poradi').cast(str)).alias('id')
    )

    prislusne_projevy = frejm.explode("poznamky_text").explode("poznamky_mluvci").filter(
            pl.col('poznamky_text').str.contains(retezec, literal=doslova) 
        |
            pl.col('poznamky_mluvci').str.contains(retezec, literal=doslova)
        ).unique(
        subset=['datum','cast','poradi']
        ).sort(by='datum')
    
    if epocha == "datum":
        
        projevu_celkem = frejm.sort(by='datum').group_by_dynamic(index_column='datum',every='1y').agg(pl.col("id").unique().len())
        
        prislusne_projevy = prislusne_projevy.group_by_dynamic(
            index_column='datum',every='1y'
        ).agg(
            pl.col("id").unique().len()
        )

    elif epocha == "obdobi":

        projevu_celkem = frejm.group_by("obdobi").agg(pl.col("id").unique().len())
        prislusne_projevy = prislusne_projevy.group_by(
            "obdobi"
        ).agg(
            pl.col("id").unique().len()
        )

    return prislusne_projevy.join(
        projevu_celkem, how="right",on=epocha
    ).with_columns(
        (pl.col("id") / pl.col("id_right")).alias("podil")
    ).sort(by=epocha)

In [12]:
zavorka('smích',epocha='datum')

id,datum,id_right,podil
u32,date,u32,f64
38,1993-01-01,9366,0.004057
21,1994-01-01,7771,0.002702
37,1995-01-01,9237,0.004006
53,1996-01-01,9568,0.005539
48,1997-01-01,16632,0.002886
54,1998-01-01,11816,0.00457
91,1999-01-01,20885,0.004357
79,2000-01-01,24941,0.003167
84,2001-01-01,20971,0.004006


In [13]:
'; '.join(df.explode("poznamky_text").group_by(
    "poznamky_text"
).len(
).sort(
    by='len',
    descending=True
).head(500).filter(pl.col('poznamky_text').is_not_null()).select(pl.col('poznamky_text')).to_series().to_list())

'potlesk.; ?; souhlas.; ano.; nesouhlasné.; nesouhlas.; potlesk zprava.; hluk v sále.; ministr: nesouhlas.; slibuji.; nesrozumitelné; předsedající: čas.; smích v sále.; ministr: souhlas.; potlesk poslanců čssd.; potlesk poslanců ods.; potlesk zleva.; upozornění na čas; potlesk poslanců ksčm.; potlesk v části sálu.; souhlasné.; potlesk části poslanců.; předsedající upozorňuje na čas.; 14.40 hodin; 14.50 hodin; ne.; 15.10 hodin; 15.20 hodin; 15.00 hodin; 15.40 hodin; upozornění na čas.; 15.30 hodin; 15.50 hodin; velký hluk v sále.; ministr nedoporučuje.; v sále je velký hluk.; 16.10 hodin; smích.; potlesk z lavic ano.; 16.20 hodin; potlesk poslanců ano.; neutrální.; 17.10 hodin; 16.00 hodin; 17.20 hodin; pobavení v sále.; 16.40 hodin; 17.30 hodin; 16.30 hodin; 10.20 hodin; 10.10 hodin; 11.00 hodin; 10.50 hodin; 17.50 hodin; 16.50 hodin; 11.40 hodin; 14.30 hodin; 17.40 hodin; 11.10 hodin; 11.20 hodin; 11.30 hodin; 10.40 hodin; 10.30 hodin; jednání zahájeno v 9.00 hodin.; 11.50 hodin; 18.0

In [14]:
def graf_vyvoje(slovo, epocha="datum"):
    if epocha == "datum":
        osa_x = 'datum:T'
    elif epocha == "obdobi":
        osa_x = "obdobi:N"
    return alt.Chart(
        zavorka(slovo,epocha=epocha).to_pandas(),
        width = 800
    ).mark_bar(
    ).encode(
        alt.X(osa_x),
        alt.Y("podil:Q")
    )

In [15]:
graf_vyvoje("smích",epocha="datum")

alt.Chart(...)

In [16]:
graf_vyvoje("hluk",epocha="obdobi")

alt.Chart(...)

In [17]:
graf_vyvoje("bučení",epocha="obdobi")

alt.Chart(...)

In [18]:
graf_vyvoje("smích",epocha="obdobi")

alt.Chart(...)

In [19]:
graf_vyvoje("potlesk",epocha="obdobi")

alt.Chart(...)

In [20]:
alt.Chart(
    zavorka("smích").to_pandas(),
    width = 800
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

alt.Chart(...)

In [21]:
alt.Chart(
    zavorka("výkřiky").to_pandas(),
    width = 800
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

alt.Chart(...)

In [22]:
df.filter(pl.col("text").str.contains("[Bb]učení"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Ministr zdravotnictví ČR David Rath""","""2002ps_stenprot_055schuz_s055136.htm""","""http://wtd.vlada.cz/scripts/detail.php?id=9214""","""Děkuji. Je to myslím docela zase signifikantní a typické, když názor vyslovený těmi, kterých se to bytostně týká, tedy lékaři zaměstnanci a zdravotním…",15,"""ps""",2002,55,136,2006-04-21,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""55. schůze""",true,true,true,null,201,[],[]
"""Poslanec David Rath""","""2006ps_stenprot_027schuz_s027049.htm""","""/sqw/detail.sqw?id=5406""","""Dámy a pánové, pane předsedající, já bych prosil, abyste poučil o slušném chování poslance ODS, aby zde nebučeli. Nejsou, jak jsou zvyklí, v chlívě. Z…",3,"""ps""",2006,27,49,2008-01-30,"""Parlament České republiky, Poslanecká sněmovna 2006 – 2010""","""27. schůze""",true,true,true,"""5406""",426,[],[]
"""Místopředseda PSP Vojtěch Pikal""","""2017ps_stenprot_040schuz_s040102.htm""","""/sqw/detail.sqw?id=6531""","""Děkuji za slovo. Protože nemáme tu britskou tradici, tak tady neprobíhalo během předchozích projevů bučení a podobně, místo toho všeobecný nezájem pří…",7,"""ps""",2017,40,102,2020-01-23,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""40. schůze""",true,true,true,"""6531""",272,"[""potlesk poslanců pirátů.""]",[]
"""Předsedkyně PSP Markéta Pekarová Adamová""","""2021ps_stenprot_005schuz_s005019.htm""","""/sqw/detail.sqw?id=6147""","""Vzhledem k tomu, že program schůze nebyl schválen a nikdo další se s přednostním právem nehlásí, tak já tuto 5. schůzi Poslanecké sněmovny končím. (Po…",2,"""ps""",2021,5,19,2021-12-15,"""PČR, PS 2021-...""","""5. schůze""",true,true,true,"""6147""",77,"[""poslanci tlučou do lavic, ozývá se bučení a volání."", ""schůze skončila ve 12.14 hodin.""]",[]
"""Místopředseda PSP Jan Skopeček""","""2021ps_stenprot_009schuz_s009170.htm""","""/sqw/detail.sqw?id=6418""","""Děkuji za slovo. Vážené kolegyně, vážení kolegové, prostřednictvím paní předsedající bych chtěl reagovat na pana předsedu Okamuru. Jsem přesvědčen, že…",3,"""ps""",2021,9,170,2022-02-18,"""PČR, PS 2021-...""","""9. schůze""",true,true,true,"""6418""",119,"[""bučení v lavicích spd."", ""potlesk z řad koalice.""]",[]
"""Poslanec Jakub Michálek""","""2021ps_stenprot_055schuz_s055007.htm""","""/sqw/detail.sqw?id=6477""","""Děkuji za slovo. Vážená paní předsedkyně, vážené kolegyně, vážení kolegové, pojďme prosím nechat v rozpravě vystoupit i ministry, například tady pana …",9,"""ps""",2021,55,7,2023-02-28,"""PČR, PS 2021-...""","""55. schůze""",true,true,true,"""6477""",148,"[""nesouhlas a bučení poslanců ano.""]",[]


In [23]:
alt.Chart(
    zavorka("bučen").to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

alt.Chart(...)

In [24]:
df.filter(pl.col("text").str.contains("(Buče",literal=True))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Místopředseda PSP Jan Skopeček""","""2021ps_stenprot_009schuz_s009170.htm""","""/sqw/detail.sqw?id=6418""","""Děkuji za slovo. Vážené kolegyně, vážení kolegové, prostřednictvím paní předsedající bych chtěl reagovat na pana předsedu Okamuru. Jsem přesvědčen, že…",3,"""ps""",2021,9,170,2022-02-18,"""PČR, PS 2021-...""","""9. schůze""",true,true,true,"""6418""",119,"[""bučení v lavicích spd."", ""potlesk z řad koalice.""]",[]


In [25]:
alt.Chart(
    zavorka("posmě[cš]",doslova=False).to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

alt.Chart(...)

In [26]:
df.filter(
    pl.col("datum").dt.year().is_between(1940,1970)
).filter(
    pl.col("text").str.contains("(Smích",literal=True)
)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]


In [27]:
alt.Chart(
    pl.concat([zavorka("smích"), zavorka("potlesk"), zavorka("křik")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

alt.Chart(...)

In [28]:
alt.Chart(
    pl.concat([zavorka("potlesk zleva"), zavorka("potlesk zprava")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

alt.Chart(...)

In [29]:
def obrat(cislo):
    if cislo >= 1:
        return cislo
    if cislo < 1:
        return -(1/cislo)

In [30]:
obrat(0.1)

-10.0

In [31]:
pravolevo = zavorka(
    "(potlesk zleva|potlesk na levici|potlesk z levé|potlesk vlevo)",doslova=False
).with_columns(
    pl.col("podil").alias("zleva")
).select(
    pl.col(['datum','zleva'])
).join(
    zavorka(
        "(potlesk zprava|potlesk na pravici|potlesk z pravé|potlesk vpravo)", doslova=False
    ).with_columns(
        pl.col("podil").alias("zprava")
    ).select(
        pl.col(['datum','zprava'])
    ), on="datum", how="full"
).with_columns(
    (pl.col('zprava') / pl.col('zleva')).map_elements(obrat, return_dtype=pl.Float32).alias('pravo-levo')
).filter(
    pl.col('datum').dt.year() >= 1993
)
pravolevo

datum,zleva,datum_right,zprava,pravo-levo
date,f64,date,f64,f32
1993-01-01,null,1993-01-01,null,null
1994-01-01,null,1994-01-01,null,null
1995-01-01,0.000325,1995-01-01,0.000108,-3.0
1996-01-01,0.000314,1996-01-01,0.000627,2.0
1997-01-01,0.001022,1997-01-01,0.000241,-4.25
1998-01-01,0.0011,1998-01-01,0.000508,-2.166667
1999-01-01,0.001724,1999-01-01,0.002059,1.194444
2000-01-01,0.000882,2000-01-01,0.001002,1.136364
2001-01-01,0.000286,2001-01-01,0.000715,2.5


## Po 9 letech se víc tleská zleva!

In [170]:
graf_do_clanku = alt.Chart(
    pravolevo.to_pandas(),
    title=["Odkud zněl ve Sněmovně potlesk"],
    width=300,
    height=250
).mark_bar(
    color="#D55F50"
).encode(
    alt.Y('datum:T', axis=alt.Axis(title=None)),
    alt.X('pravo-levo:Q',axis=alt.Axis(title="← víc se tleská zleva ~ víc se tleská zprava →", orient="top", labelExpr="datum.label + '×'"))
).configure_axisX(
    grid=True, domain=False
).configure_axisY(
    grid=True, domain=False
).configure_view(
    stroke='transparent'
).resolve_axis(
    x="independent",
    y="independent"
).resolve_scale(
    y="shared"
)

graf_do_clanku

alt.Chart(...)

In [172]:
me_to_neurazi(graf=graf_do_clanku, soubor='odkud_potlesk', kredity='vizualizace: iROZHLAS.cz | zdroj projevů: psp.cz', slozka_na_serveru='grafy_stenozaznamy')

<figure>
    <a href="https://data.irozhlas.cz/grafy_stenozaznamy/odkud_potlesk.svg" target="_blank">
    <img src="https://data.irozhlas.cz/grafy_stenozaznamy/odkud_potlesk.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [34]:
pravolevo_obdobi = zavorka(
    "(potlesk zleva|potlesk na levici|potlesk z levé|potlesk vlevo)", epocha="obdobi",doslova=False
).with_columns(
    pl.col("podil").alias("zleva")
).select(
    pl.col(['obdobi','zleva'])
).join(
    zavorka(
        "(potlesk zprava|potlesk na pravici|potlesk z pravé|potlesk vpravo)", epocha="obdobi", doslova=False
    ).with_columns(
        pl.col("podil").alias("zprava")
    ).select(
        pl.col(['obdobi','zprava'])
    ), on="obdobi", how="full"
).with_columns(
    (pl.col('zprava') / pl.col('zleva')).map_elements(obrat, return_dtype=pl.Float32).alias('pravo-levo')
)
pravolevo_obdobi

obdobi,zleva,obdobi_right,zprava,pravo-levo
i32,f64,i32,f64,f32
1993,0.0001,1993,0.000033,-3.0
1996,0.000877,1996,0.000455,-1.928571
1998,0.000905,1998,0.001128,1.246575
2002,0.000791,2002,0.003274,4.138462
2006,0.005086,2006,0.004946,-1.028169
2010,0.012582,2010,0.006162,-2.041958
2013,0.00415,2013,0.011317,2.727011
2017,0.001177,2017,0.007105,6.037383
2021,0.005042,2021,0.004077,-1.236686


In [35]:
alt.Chart(
    pravolevo_obdobi.to_pandas(),
    title=["„Potlesk zprava“ (> 0) a „potlesk zleva“ (< 0)","ve stenozáznamech PS PSP ČR"]
).mark_bar().encode(alt.Y('obdobi:N'),alt.X('pravo-levo:Q',axis=alt.Axis(title="<– víc se tleská zleva | víc se tleská zprava ->")))

alt.Chart(...)

## Počkat: kdo sedí vpravo, a kdo vlevo?

In [37]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Andrej Babiš'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

5

In [38]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Andrej Babiš'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

28

In [39]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Petr Fiala'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

8

In [40]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Petr Fiala'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

0

In [41]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Tomio Okamura'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

2

In [42]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Tomio Okamura'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

10

In [43]:
len(df.filter(
    (pl.col('mluvci').str.contains('Grebeníček'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

13

In [44]:
len(df.filter(
    (pl.col('mluvci').str.contains('Grebeníček'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

0

In [45]:
len(df.filter(
    (pl.col('mluvci').str.contains('Miroslav Sládek'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

0

In [46]:
len(df.filter(
    (pl.col('mluvci').str.contains('Miroslav Sládek'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

9

In [47]:
len(df.filter(
    (pl.col('mluvci').str.contains('Bohuslav Sobotka'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

47

In [48]:
len(df.filter(
    (pl.col('mluvci').str.contains('Bohuslav Sobotka'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

0

In [49]:
alt.Chart(
    pl.concat([zavorka("pobavení"), zavorka("veselost"), zavorka("veselí")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

alt.Chart(...)

In [50]:
alt.Chart(
    pl.concat([zavorka("\(Hluk"), zavorka("\(Ticho")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

alt.Chart(...)

In [51]:
alt.Chart(
    pl.concat([zavorka("(Smích", doslova=True), zavorka("(Veselí", doslova=True), zavorka("(Pobavení", doslova=True)]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

alt.Chart(...)

In [52]:
alt.Chart(
    df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len()).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('text:Q', axis=alt.Axis(title=None)),
)

alt.Chart(...)

In [53]:
df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len())

datum,text
date,u32
1993-01-01,9337
1994-01-01,7730
1995-01-01,9219
1996-01-01,9345
1997-01-01,16367
1998-01-01,11729
1999-01-01,20383
2000-01-01,23721
2001-01-01,20242


In [54]:
zebricek_potlesku = df.filter(
    pl.col("obdobi") == 2021
).explode(
    "poznamky_text"
).filter(
    pl.col("poznamky_text").str.contains("potlesk")
).group_by(
    "mluvci"
).len(
).sort(
    by="len",descending=True
)

zebricek_potlesku

mluvci,len
str,u32
"""Poslankyně Alena Schillerová""",592
"""Poslanec Andrej Babiš""",393
"""Poslanec Tomio Okamura""",199
"""Místopředseda PSP Karel Havlíček""",179
"""Poslanec Radek Vondráček""",147
"""Poslanec Patrik Nacher""",142
"""Poslanec Radim Fiala""",138
"""Poslankyně Berenika Peštová""",131
"""Poslanec Robert Králíček""",112


In [55]:
df.filter(
    pl.col("obdobi") == 2010
).explode(
    "poznamky_text"
).filter(
    pl.col("poznamky_text").str.contains("smích") 
    | 
    pl.col("poznamky_text").str.contains("pobavení") 
    | 
    pl.col("poznamky_text").str.contains("veselí") 
    | 
    (~pl.col("poznamky_text").str.contains("smíchem"))
).group_by(
    "mluvci"
).len(
).sort(
    by="len",descending=True
)

mluvci,len
str,u32
"""Místopředseda PSP Lubomír Zaorálek""",1868
"""Předsedkyně PSP Miroslava Němcová""",1844
"""Místopředsedkyně PSP Vlasta Parkanová""",531
"""Místopředsedkyně PSP Kateřina Klasnová""",476
"""Místopředseda PSP Jiří Oliva""",394
"""Poslanec Bohuslav Sobotka""",292
"""Ministr financí ČR Miroslav Kalousek""",251
"""Poslanec Jeroným Tejc""",220
"""Poslanec David Rath""",188


In [56]:
zebricek_smichu = df.filter(
    pl.col("obdobi") == 2021
).explode(
    "poznamky_text"
).filter(
    pl.col("poznamky_text").str.contains("smích") 
    | 
    pl.col("poznamky_text").str.contains("pobavení") 
    | 
    pl.col("poznamky_text").str.contains("veselí")
).group_by(
    "mluvci"
).len(
).sort(
    by="len",descending=True
)

zebricek_smichu

mluvci,len
str,u32
"""Poslanec Andrej Babiš""",321
"""Poslanec Tomio Okamura""",92
"""Místopředseda PSP Karel Havlíček""",72
"""Poslanec Patrik Nacher""",66
"""Místopředsedkyně PSP Věra Kovářová""",52
"""Místopředseda PSP Jan Skopeček""",47
"""Poslankyně Berenika Peštová""",45
"""Poslanec Radek Vondráček""",40
"""Poslanec Jakub Michálek""",38


In [57]:
zebricek_slov = df.filter(
    pl.col("obdobi") == 2021
).group_by(
    "mluvci"
).agg(
    pl.col("pocet_slov").sum()
)

zebricek_slov

mluvci,pocet_slov
str,u32
"""Poslanec Patrik Nacher""",391630
"""Senátor Josef Bazala""",887
"""Senátor Lumír Kantor""",630
"""Ministryně obrany ČR Jana Černochová""",94933
"""Senátor Jan Holásek""",421
"""Poslanec Jiří Navrátil""",16492
"""Poslankyně Klára Kocmanová""",32910
"""Poslanec Jan Jakob""",43713
"""Senátor Jan Paparega""",602


In [58]:
zebricek_smichu.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).drop_nulls(
).sort(
    by="podil",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
1,"""Poslanec Josef Váňa""",122,0.008197
6,"""Prezident České republiky Miloš Zeman""",2533,0.002369
1,"""Senátor Ladislav Václavec""",532,0.00188
10,"""Poslanec Vít Vomáčka""",13931,0.000718
1,"""Poslanec Jan Farský""",1434,0.000697
1,"""Senátor Zdeněk Nytra""",1499,0.000667
1,"""Poslanec Zbyněk Stanjura""",1752,0.000571
15,"""Poslanec Jaroslav Foldyna""",28874,0.000519
4,"""Poslanec Bohuslav Svoboda""",8043,0.000497


In [59]:
df.filter(pl.col("obdobi") == 2021).filter(pl.col("mluvci").str.contains("Josef Bernard")).filter(pl.col("text").str.contains("směj"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]


In [60]:
zebricek_smichu.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).drop_nulls(
).filter(
    pl.col("pocet_slov") > 10000
).sort(
    by="podil",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
10,"""Poslanec Vít Vomáčka""",13931,0.000718
15,"""Poslanec Jaroslav Foldyna""",28874,0.000519
8,"""Poslanec Aleš Dufek""",22142,0.000361
12,"""Ministr spravedlnosti ČR Pavel Blažek""",36491,0.000329
321,"""Poslanec Andrej Babiš""",1095221,0.000293
24,"""Poslanec Richard Brabec""",90254,0.000266
6,"""Poslanec Jan Volný""",24825,0.000242
4,"""Poslanec Karel Sládeček""",16648,0.00024
14,"""Poslanec Milan Brázdil""",58974,0.000237


In [61]:
df.filter(pl.col("mluvci").str.contains("Ivan Bartoš")).select(pl.col("pocet_slov").sum())

pocet_slov
u32
347395


In [62]:
df.filter(pl.col("mluvci").str.contains("Ivan Bartoš")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("směj"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,list[str]
"""Místopředseda vlády a ministr pro místní rozvoj ČR Ivan Bartoš""","""2021ps_stenprot_019schuz_s019175.htm""","""https://www.vlada.cz/cz/clenove-vlady/ivan-bartos-191704/""","""(Směje se.) Já jsem rád, že to takhle řešíme. Tak uděláme analýzu, které nejdůležitější legislativy kdy šly v předchozím volebním období poslaneckým n…",2,"""ps""",2021,19,175,2022-05-11,"""PČR, PS 2021-...""","""19. schůze""",true,true,true,null,91,"""směje se.""",[]


In [63]:
zebricek_smichu.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).filter(
    pl.col("len").is_null()
).sort(
    by="pocet_slov",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
null,"""Poslanec Vladimír Zlínský""",114388,null
null,"""Poslanec Igor Hendrych""",89885,null
null,"""Poslankyně Marie Pošarová""",70868,null
null,"""Poslankyně Klára Dostálová""",68324,null
null,"""Poslanec Radek Rozvoral""",64222,null
null,"""Poslankyně Ivana Mádlová""",57931,null
null,"""Poslankyně Jana Hanzlíková""",56081,null
null,"""Poslankyně Vladimíra Lesenská""",53453,null
null,"""Poslanec Josef Kott""",53022,null


In [64]:
df.filter(pl.col("obdobi") == 2021).filter(pl.col("mluvci").str.contains("Vladimír Zlínský")).filter(pl.col("text").str.contains("(?i)smích"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]


In [65]:
df.filter(pl.col("obdobi") == 2021).filter(pl.col("mluvci").str.contains("Vít Vomáčka")).filter(pl.col("text").str.contains("(?i)smích"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Poslanec Vít Vomáčka""","""2021ps_stenprot_009schuz_s009223.htm""","""/sqw/detail.sqw?id=6792""","""Musím zareagovat na pana kolegu. Nevím, jak dlouho jste starostoval, já starostuju jednatřicet let. Myslím si, že Kravaře, policie říká o tom, že Krav…",7,"""ps""",2021,9,223,2022-03-03,"""PČR, PS 2021-...""","""9. schůze""",true,true,true,"""6792""",64,"[""smích z lavic ano."", ""oživení a potlesk zleva.""]",[]
"""Poslanec Vít Vomáčka""","""2021ps_stenprot_079schuz_s079077.htm""","""/sqw/detail.sqw?id=6792""","""Vážení páni ministři, dámy a pánové, přiznám se, že nevystupuji, abych reagoval na svého ctěného pana kolegu, ale vystupuji kvůli tomu, že jsem tady n…",5,"""ps""",2021,79,77,2023-10-17,"""PČR, PS 2021-...""","""79. schůze""",true,true,true,"""6792""",120,"[""smích několika poslanců.""]",[]
"""Poslanec Vít Vomáčka""","""2021ps_stenprot_088schuz_s088077.htm""","""/sqw/detail.sqw?id=6792""","""Děkuji za slovo, vážený pane předsedající. Vážení páni ministři, dámy a pánové, já jsem se hlásil o slovo, když tady to začalo jiskřit mezi panem Kolo…",15,"""ps""",2021,88,77,2024-01-17,"""PČR, PS 2021-...""","""88. schůze""",true,true,true,"""6792""",137,"[""pobavení v sále."", ""smích v sále.""]",[]
"""Poslanec Vít Vomáčka""","""2021ps_stenprot_088schuz_s088232.htm""","""/sqw/detail.sqw?id=6792""","""Děkuji za slovo. Vážená paní předsedající, dámy a pánové, já jsem tady minule mluvil o tom, že tady vznikají určité dvojice, a přiznám se, že vždycky …",3,"""ps""",2021,88,232,2024-01-23,"""PČR, PS 2021-...""","""88. schůze""",true,true,true,"""6792""",136,"[""smích v levé části jednacího sálu."", ""smích v levé části jednacího sálu, malý potlesk v pravé části sálu."", ""pobavení v jednacím sále.""]",[]
"""Poslanec Vít Vomáčka""","""2021ps_stenprot_113schuz_s113082.htm""","""/sqw/detail.sqw?id=6792""","""Takže potřetí, paní Peštová, děkuju, že do této doby... Ale vidím, že se hlásíte, tak to asi začne střílet, ale já odstupuju, protože na vás nemám. Dě…",1,"""ps""",2021,113,82,2024-10-03,"""PČR, PS 2021-...""","""113. schůze""",true,true,false,"""6792""",31,"[""smích v levé části sálu.""]",[]


In [66]:
zebricek_potlesku.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).filter(
    pl.col("len").is_null()
).sort(
    by="pocet_slov",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
null,"""Poslanec Josef Kott""",53022,null
null,"""Ministr školství, mládeže a tělovýchovy ČR Mikuláš Bek""",49888,null
null,"""Poslanec David Štolpa""",17001,null
null,"""Ministr průmyslu a obchodu ČR Lukáš Vlček""",16854,null
null,"""Ministr pro místní rozvoj ČR Petr Kulhánek""",15043,null
null,"""Poslanec Matěj Ondřej Havel""",13488,null
null,"""Ministr školství, mládeže a tělovýchovy ČR Vladimír Balaš""",12668,null
null,"""Poslanec Petr Bendl""",12321,null
null,"""Poslanec Michael Rataj""",10725,null


In [67]:
zebricek_potlesku.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).drop_nulls(
).filter(
    pl.col("pocet_slov") > 31000
).sort(
    by="podil",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
112,"""Poslanec Robert Králíček""",79673,0.001406
38,"""Poslankyně Renata Oulehlová""",31971,0.001189
592,"""Poslankyně Alena Schillerová""",511438,0.001158
33,"""Poslanec Milan Feranec""",34981,0.000943
44,"""Poslanec Marek Benda""",49888,0.000882
29,"""Poslanec Tomáš Kohoutek""",34246,0.000847
23,"""Ministr spravedlnosti ČR Pavel Blažek""",36491,0.00063
41,"""Poslanec Kamal Farhan""",67206,0.00061
147,"""Poslanec Radek Vondráček""",245689,0.000598


In [68]:
df.filter(pl.col("mluvci").str.contains("Robert Králíček")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("potlesk")).group_by("text").len().sort(by="len")

text,len
str,u32
"""Zpátky ale k samotné digitalizaci. Systém řízení byl pouze iluzí. Rozhodnutí nebyla zaznamenávána, dokumentace scházela a odpovědné osoby nebyly ident…",1
"""Už budu velmi krátký. Už jsem to udělal osobně, ale chci to udělat veřejně. Panu Coganovi se prostřednictvím předsedajícího omlouvám. Můj výrok byl ne…",1
"""Vážený pane místopředsedo, vážená vládo, kolegyně kolegové, mně se strašně líbí, jak pan ministr Síkela přijde pozdě s něčím a ještě se tu pokouší zne…",1
"""Vážená paní předsedkyně, kolegyně, kolegové, použiji oblíbené Nacherovo slovo ""já jsem nechtěl vystupovat"". Ale chtěl bych reagovat na pana Ondřeje Ba…",1
"""Pokud už nebyla vláda schopna vetovat pakt jako celek, měla aspoň vyjednat výjimku z relokační povinnosti. Velkou příležitost k tomu měla, byla předse…",1
"""Vážený pane předsedající, vážený pane ministře, vážené kolegyně, vážení kolegové, když jsme před časem začali projednávat zákon o dostupném bydlení, d…",1
"""Digitální a informační agentura si už na příští rok, kdy vynaloží 40 milionů na eDokladovku, taktéž rozpočtuje 15 milionů už na evropskou digitální pe…",1
"""Já bych také začal řečnickou otázkou, jestli by pan Bauer prosadil zákon bez podpory tehdejší koalice, a to je právě ten rozdíl. Vy jste byl, pane min…",1
"""Ale mě by zajímal názor například místopředsedkyně Olgy Richterové na kandidaturu pana Poledníčka, protože Piráti byli ti, kteří nám říkali, že oni ta…",1


In [69]:
df.filter(pl.col("mluvci").str.contains("Josef Váňa"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Poslanec Josef Váňa""","""2021ps_stenprot_117schuz_s117088.htm""","""/sqw/detail.sqw?id=6946""","""Děkuji za slovo, paní předsedající. Dobrý den, kolegyně, kolegové, já bych se chtěl zeptat: ti, co navrhli tady tohle, kam jste zařadili koňaře? (Smíc…",6,"""ps""",2021,117,88,2024-11-08,"""PČR, PS 2021-...""","""117. schůze""",true,true,true,"""6946""",113,"[""smích v sále."", ""potlesk vlevo."", ""potlesk vlevo.""]",[]
"""Poslanec Josef Váňa""","""2021ps_stenprot_127schuz_s127021.htm""","""/sqw/detail.sqw?id=6946""","""Na sjetině mám pro, ale jsem proti, a zpochybňuji hlasování.""",4,"""ps""",2021,127,21,2025-01-21,"""PČR, PS 2021-...""","""127. schůze""",true,true,true,"""6946""",9,[],[]


In [70]:
df.filter(pl.col("mluvci").str.contains("Josef Váňa")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("potlesk")).group_by("text").len().sort(by="len")

text,len
str,u32
"""Děkuji za slovo, paní předsedající. Dobrý den, kolegyně, kolegové, já bych se chtěl zeptat: ti, co navrhli tady tohle, kam jste zařadili koňaře? (Smíc…",2


In [71]:
df.filter(pl.col("mluvci").str.contains("Josef Váňa")).head(1).select(pl.col("soubor")).item()

'2021ps_stenprot_117schuz_s117088.htm'

In [72]:
df.filter(pl.col("mluvci").str.contains("Josef Váňa")).head(1).select(pl.col("text")).item()

'Děkuji za slovo, paní předsedající. Dobrý den, kolegyně, kolegové, já bych se chtěl zeptat: ti, co navrhli tady tohle, kam jste zařadili koňaře? (Smích v sále.) Prosím vás, tady tohle je povolání, které může dělat jenom blázen, to znamená soboty, neděle, dvanáct hodin denně. A řeknu vám hroznou skutečnost - dneska v šedesáti neznám jediného člověka, který by pracoval u koní, a nebyl invalidní důchodce nebo neměl zdravotní problémy. (Potlesk vlevo.) Moje paní má 57 let, a když jsem přišel domů a řekl jsem jí, že půjde za deset let do důchodu, tak mě málem vyhnala z baráku. Je to hrozné, ale prostě mně se ta vaše důchodová reforma vůbec nelíbí. Děkuji. (Potlesk vlevo.)'